In [60]:
import pandas as pd
import string
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
import pickle

In [7]:
training_data = pd.read_csv('trainingandtestdata/training.1600000.processed.noemoticon.csv', header=None, names=['Sentiment', 'ID', 'date', 'query', 'user', 'tweet'])

In [5]:
test_data = pd.read_csv('trainingandtestdata/testdata.manual.2009.06.14.csv', header=None, names=['Sentiment', 'ID', 'date', 'query', 'user', 'tweet'])

In [67]:
print len(training_data)
print len(test_data)

1600000
498


In [51]:
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['RT', 'rt', 'via', 'http', 'https', ':/']

def word_feats(words):
    return dict([(word, True) for word in words.split(' ') if word not in stop])

training_data['feats'] = training_data['tweet'].apply(word_feats)
test_data['feats'] = test_data['tweet'].apply(word_feats)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:5: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [52]:
training_data.head()

,Sentiment,ID,date,query,user,tweet,feats
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","{u'': True, u'it.': True, u';D': True, u'Third..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,"{u'': True, u'School': True, u'texting': True,..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,"{u'': True, u'50%': True, u'Managed': True, u'..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"{u'body': True, u'': True, u'like': True, u'fi..."
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","{u'no,': True, u'behaving': True, u'there.': T..."


In [53]:
test_data.head()

,Sentiment,ID,date,query,user,tweet,feats
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...,"{u'fantastic': True, u'I': True, u'Kindle2.': ..."
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...,"{u'': True, u'childs': True, u'Love': True, u'..."
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck...","{u'assesment': True, u'fucking': True, u'rocks..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...,"{u'back.': True, u'months': True, u'love': Tru..."
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...,"{u'': True, u'perfect': True, u'@mikefish': Tr..."


In [54]:
training_data.to_pickle('pkl/nb_train.pkl')
test_data.to_pickle('pkl/nb_test.pkl')

In [58]:
def transform_feat(df):
    
    feats = []
    
    for r in df.iterrows():
        sent = r[1][0]
        feat = r[1][6]
        if sent == 0:
            feats.append((feat, 'neg'))
        elif sent == 4:
            feats.append((feat, 'pos'))
        
        
    return feats

In [59]:
trainfeats = transform_feat(training_data)
testfeats = transform_feat(test_data)

In [64]:
with open("pkl/trainfeats.pkl", "wb") as f:
    pickle.dump(trainfeats, f)
    
with open("pkl/testfeats.pkl", "wb") as f:
    pickle.dump(testfeats, f)

In [66]:
classifier = NaiveBayesClassifier.train(trainfeats)
print 'accuracy:', nltk.classify.util.accuracy(classifier, testfeats)
classifier.show_most_informative_features()

accuracy: 0.810584958217
Most Informative Features
                   Died! = True              neg : pos    =    140.3 : 1.0
                 me..its = True              pos : neg    =    114.3 : 1.0
                 bummed. = True              neg : pos    =     97.0 : 1.0
                 McMahon = True              neg : pos    =     76.3 : 1.0
                     447 = True              neg : pos    =     74.3 : 1.0
                  Farrah = True              neg : pos    =     72.3 : 1.0
             @Banksyart2 = True              pos : neg    =     61.7 : 1.0
                   Boooo = True              neg : pos    =     53.0 : 1.0
                 *cries* = True              neg : pos    =     51.0 : 1.0
                    sadd = True              neg : pos    =     49.0 : 1.0


In [69]:
classifier.labels()

['neg', 'pos']

In [ ]:
with open('pkl/NB.pkl', 'w') as f:
    pickle.dump(classifier, f)